In [2]:
#The imports
from sklearn.cluster import SpectralClustering
import numpy as np
import pandas as pd

In [102]:
#Read the parameters of the fitted nested variogram structures, wich were saved as a GEO-EAS point set.
# 1st column: parameter a (main range)
# 2nd column: parameter b/a (minor/main ranges ratio)
# 3rd column: parameter theta (azimuth)
# 4th column: parameter cc (contribution)

varParameters = pd.read_csv("Scat_V4_GA_FIM_816.xyz", sep='\t', skiprows=0)
print(varParameters.values.shape)

numberOfClusters = 4


(816, 4)


In [103]:
#Perform the clustering using only range, ranges ratio and azimuth (does not use contribution)
#clusteringData = SpectralClustering(n_clusters=numberOfClusters, 
#                                    assign_labels="discretize", 
#                                    random_state=0).fit( varParameters.values[:,:3] )

#Perform the clustering using all variables
clusteringData = SpectralClustering(n_clusters=numberOfClusters, 
                                    assign_labels="discretize", 
                                    random_state=0).fit( varParameters )

In [104]:
#Stores the mean parameters for each cluster
rangeMeans       = np.zeros( numberOfClusters )
rangeRatioMeans  = np.zeros( numberOfClusters )
azimuthMeans     = np.zeros( numberOfClusters )
ccMeans          = np.zeros( numberOfClusters )

#The counts of samples for each cluster 
sampleCounts     = np.zeros( numberOfClusters )

#Accumulate the sums of parameters per cluster for the computation of the means (cluster centers)
for iRow in range(0, clusteringData.labels_.shape[0] ) :
    rangeV                 = varParameters.values[ iRow, 0 ]
    rangeRatio             = varParameters.values[ iRow, 1 ]
    azimuth                = varParameters.values[ iRow, 2 ]
    covarianceContribution = varParameters.values[ iRow, 3 ]
    
    clusterId = clusteringData.labels_[ iRow ]
    
    rangeMeans     [ clusterId ] += rangeV 
    rangeRatioMeans[ clusterId ] += rangeRatio 
    azimuthMeans   [ clusterId ] += azimuth 
    ccMeans        [ clusterId ] += covarianceContribution 

    sampleCounts   [ clusterId ] += 1
   
#Compute the means of parameters per cluster (cluster centers)
for iCluster in range(0, numberOfClusters) :
    rangeMeans     [ iCluster ] /= sampleCounts [ iCluster ] 
    rangeRatioMeans[ iCluster ] /= sampleCounts [ iCluster ] 
    azimuthMeans   [ iCluster ] /= sampleCounts [ iCluster ] 
    ccMeans        [ iCluster ] /= sampleCounts [ iCluster ] 
    
    print("Cluster", iCluster+1, ":")
    print("  a=", rangeMeans[iCluster], 
          "b/a=", rangeRatioMeans[iCluster], 
          "az=", azimuthMeans[iCluster], 
          "cc=", ccMeans[iCluster])
 


Cluster 1 :
  a= 9.492709617569137 b/a= 0.22910454927928248 az= 135.17230869355834 cc= 0.15927626034723769
Cluster 2 :
  a= 51.16918495695807 b/a= 0.4760274863478581 az= 98.17017908467098 cc= 0.11852751166136129
Cluster 3 :
  a= 15.594723655051975 b/a= 0.33922128708597465 az= 51.19707721427509 cc= 0.12059519846485309
Cluster 4 :
  a= 50.48868250768054 b/a= 0.6828502896937856 az= 77.57213443502107 cc= 0.20445952429760966
